In [2]:
# ======================================================
# Noise Schedule Utilities
# ======================================================

import torch
import matplotlib.pyplot as plt

# -------------------------
# 1) Define Noise Schedule
# -------------------------
def make_beta_schedule(T, beta_start=1e-4, beta_end=0.02):
    """
    Linear beta schedule exactly as in original DDPM code
    """
    return torch.linspace(beta_start, beta_end, T)

def build_noise_schedule(T, device="cpu"):
    """
    Returns betas, alphas, alpha_bars
    Compatible with main diffusion training code
    """
    betas = make_beta_schedule(T).to(device)
    alphas = (1.0 - betas).to(device)
    alpha_bars = torch.cumprod(alphas, dim=0).to(device)
    return betas, alphas, alpha_bars


# -------------------------
# 2) Inspect Schedule
# -------------------------
def print_schedule_samples(betas, alphas, alpha_bars, steps=None):
    """
    Print selected timesteps from the schedule. If steps is None, use a sensible default
    and ensure all indices are within the valid range.
    """
    n = betas.shape[0]
    if steps is None:
        default = [0,1,2,3,4,5,6,7,8,9,10,50,100,200,300,400,n-1]
        steps = [t for t in default if 0 <= t < n]
    print(" t |    beta    |   alpha   | alpha_bar")
    print("----------------------------------------")
    for t in steps:
        print(f"{t:3d} | {betas[t]:.6f} | {alphas[t]:.6f} | {alpha_bars[t]:.6f}")


# -------------------------
# 3) Quick Test Run
# -------------------------
if __name__ == "__main__":
    T = 500
    device = "cuda" if torch.cuda.is_available() else "cpu"

    betas, alphas, alpha_bars = build_noise_schedule(T, device)

    print_schedule_samples(betas, alphas, alpha_bars)


 t |    beta    |   alpha   | alpha_bar
----------------------------------------
  0 | 0.000100 | 0.999900 | 0.999900
  1 | 0.000140 | 0.999860 | 0.999760
  2 | 0.000180 | 0.999820 | 0.999580
  3 | 0.000220 | 0.999780 | 0.999361
  4 | 0.000260 | 0.999740 | 0.999101
  5 | 0.000299 | 0.999701 | 0.998802
  6 | 0.000339 | 0.999661 | 0.998464
  7 | 0.000379 | 0.999621 | 0.998085
  8 | 0.000419 | 0.999581 | 0.997667
  9 | 0.000459 | 0.999541 | 0.997209
 10 | 0.000499 | 0.999501 | 0.996711
 50 | 0.002094 | 0.997906 | 0.945552
100 | 0.004088 | 0.995912 | 0.809140
200 | 0.008076 | 0.991924 | 0.438713
300 | 0.012064 | 0.987936 | 0.159121
400 | 0.016052 | 0.983948 | 0.038544
499 | 0.020000 | 0.980000 | 0.006353


الصورة الأصلية (بدون ضوضاء) قيمتها = 1

عند الزمن t = 1 بنضيف ضوضاء مقدارها
β (beta) = 0.000100

إذن الجزء اللي ضاع بسبب الضوضاء = beta

والجزء اللي لسه من الصورة =
α (alpha) = 1 − beta = 0.999100

المعاني بشكل واضح:

beta (β)
👉 كمية الـ noise اللي بتتضاف في خطوة واحدة (step واحدة)

alpha (α)
👉 قد إيه لسه باقي من الصورة الأصلية بعد إضافة الـ noise في خطوة واحدة (عند t معيّن)

alphā (alpha_bar)
👉 قد إيه باقي من الصورة الأصلية بعد عدد خطوات t
(يعني التأثير التراكمي لكل الـ noise اللي اتضافت لحد اللحظة دي)

الفكرة الكبيرة 🧠

كل خطوة بتاكل شوية صغيرة من الصورة

alpha = اللي باقي بعد خطوة واحدة

alpha_bar = اللي باقي بعد خطوات كتير

ومع زيادة t →
الصورة بتختفي تدريجيًا → وتتحول لضوضاء


---

## What does each term represent?

- **beta (βₜ)**  
  The amount of **new noise added** at timestep `t`.

- **alpha (αₜ = 1 − βₜ)**  
  The amount of **signal preserved** at timestep `t`.

- **alpha_bar (ᾱₜ)**  
  The **total remaining fraction of the original image** after `t` steps.

---

## Interpreting the Results

### 🔹 At t = 0
- `alpha_bar ≈ 0.9999`
- Almost the entire original image is still visible.
- Very tiny amount of noise added.

---

###  At t = 50
- `alpha_bar ≈ 0.9455`
- About **94% of the original image remains**.
- Noise is present but image structure is still clear.

---

###  At t = 100
- `alpha_bar ≈ 0.8091`
- About **81% of the image remains**.
- Noise becomes clearly noticeable.

---

###  At t = 200
- `alpha_bar ≈ 0.4387`
- Less than **50% of the original image remains**.
- Image is heavily corrupted by noise.

---

### At t = 499 (final step)
- `alpha_bar ≈ 0.0063`
- Less than **1% of original signal remains**.
- The image is now **pure noise**.

---